# rating-gp (prototype)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr

%matplotlib inline

In [ ]:
# setup

# SF Coeur D Alene River 
site = "12413470"
start_date = "1988-10-01" 
end_date = "2021-09-30" 

In [ ]:
from rating_gp.providers.usgs import get_measurements

In [ ]:
training_data = get_measurements(site=site, start_date=start_date, end_date=end_date)

In [ ]:
training_data

In [ ]:
%%time
# select an engine
from rating_gp.models.gpytorch import RatingGPMarginalGPyTorch as RatingGP




model = RatingGP()
model.fit(target=training_data['discharge'], covariates=training_data[['stage']])